In [ ]:
import os
os.chdir("/home/elicer/ptta") # os.chdir("/home/ubuntu/test-time-adapters")

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)

In [ ]:
import sys
from os import path, environ
from argparse import ArgumentParser

import torch

from ttadapters import datasets, models
from ttadapters.models.base import ModelProvider
from ttadapters.utils import visualizer, validator
from ttadapters.datasets import DatasetHolder, scenarios

from ttadapters.methods.other_method.baseline import ActMADConfig, ActMAD, NORMConfig, NORM, DUAConfig, DUA, MeanTeacherConfig, MeanTeacher, WHWConfig, WHW

In [ ]:
environ["TORCHDYNAMO_CAPTURE_SCALAR_OUTPUTS"] = "1"
environ["TORCHDYNAMO_CAPTURE_DYNAMIC_OUTPUT_SHAPE_OPS"] = "1"

torch._dynamo.config.capture_scalar_outputs = True
torch._dynamo.config.suppress_errors = True

In [ ]:
# Set Batch Size
BATCH_SIZE = 2, 8, 4  # Local
#BATCH_SIZE = 40, 200, 1  # A100 or H100
ACCUMULATE_STEPS = 1

# Set Data Root
DATA_ROOT = path.join(".", "data")

# Set Target Dataset
SOURCE_DOMAIN = datasets.SHIFTDataset

# Set Run Mode
TEST_MODE = True

# Set Model List
MODEL_ZOO = ["rcnn", "swinrcnn", "yolo11", "rtdetr"]
MODEL_TYPE = MODEL_ZOO[0]

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0
DATA_TYPE = torch.float32

if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        torch.cuda.set_device(DEVICE_NUM)
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))
print(f"INFO: Using data precision - {DATA_TYPE}")

In [ ]:
# Fast download patch
datasets.patch_fast_download_for_object_detection()

In [ ]:
# Basic pre-training dataset
match SOURCE_DOMAIN:
    case datasets.SHIFTDataset:
        dataset = DatasetHolder(
            train=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, train=True),
            valid=datasets.SHIFTClearDatasetForObjectDetection(root=DATA_ROOT, valid=True),
            test=datasets.SHIFTCorruptedDatasetForObjectDetection(root=DATA_ROOT, valid=True)
        )
    case datasets.CityscapesDataset:
        pass
    case _:
        raise ValueError(f"Unsupported dataset: {SOURCE_DOMAIN}")

# Dataset info
CLASSES = dataset.train.classes
NUM_CLASSES = len(CLASSES)
print(f"INFO: Number of classes - {NUM_CLASSES} {CLASSES}")

In [ ]:
# Initialize model
match MODEL_TYPE:
    case "rcnn":
        model = models.FasterRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_OFFICIAL), strict=False)
    case "swinrcnn":
        model = models.SwinRCNNForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR_NATUREYOO if TEST_MODE else model.Weights.IMAGENET_XIAOHU2015), strict=False)
    case "rtdetr":
        model = models.RTDetrForObjectDetection(dataset=SOURCE_DOMAIN)
        load_result = model.load_from(**vars(model.Weights.SHIFT_CLEAR if TEST_MODE else model.Weights.COCO_OFFICIAL), strict=False)
    case _:
        raise ValueError(f"Unsupported model type: {MODEL_TYPE}")

print("INFO: Model state loaded -", load_result)
model.to(device)

In [ ]:
config = ActMADConfig()
config.data_root = DATA_ROOT
config.lr = 0.0001
config.clear_dataset = model.DataPreparation(dataset.train, evaluation_mode=True)
adaptive_model = ActMAD(model, config)

In [ ]:
config = NORMConfig()
config.data_root = DATA_ROOT
adaptive_model = NORM(model, config)

In [ ]:
config = DUAConfig()
config.data_root = DATA_ROOT
adaptive_model = DUA(model, config)

In [ ]:
config = MeanTeacherConfig()
config.data_root = DATA_ROOT
config.lr = 0.0001
adaptive_model = MeanTeacher(model, config)

In [ ]:
config = WHWConfig()
config.data_root = DATA_ROOT
config.lr = 0.0001
config.clear_dataset = model.DataPreparation(dataset.train, evaluation_mode=True)
adaptive_model = WHW(model, config)

In [ ]:
# # Compile model
# adaptive_model = torch.compile(adaptive_model)

In [ ]:
# Ensure split (required due to Scenario class works with coroutines)
_ = datasets.SHIFTContinuousSubsetForObjectDetection(root=DATA_ROOT, train=True)

In [ ]:
data_preparation = model.DataPreparation(datasets.base.BaseDataset(), evaluation_mode=True)

discrete_scenario = scenarios.SHIFTDiscreteScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTDiscreteScenario.WHWPAPER, transforms=data_preparation.transforms
)
continuous_scenario = scenarios.SHIFTContinuousScenario(
    root=DATA_ROOT, valid=True, order=scenarios.SHIFTContinuousScenario.DEFAULT, transforms=data_preparation.transforms
)

In [ ]:
evaluator = validator.DetectionEvaluator(adaptive_model, no_grad=False, classes=CLASSES, data_preparation=data_preparation, dtype=DATA_TYPE, device=device)
evaluator_loader_params = dict(batch_size=BATCH_SIZE[2], shuffle=False, collate_fn=data_preparation.collate_fn)

In [ ]:
adaptive_model.model_provider = model.model_provider

In [ ]:
visualizer.visualize_metrics(discrete_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))

In [ ]:
visualizer.visualize_metrics(continuous_scenario(**evaluator_loader_params).play(evaluator, index=[adaptive_model.__class__.__name__]))